In [ ]:
# Including the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [ ]:
# Import data, make a copy of the original

df0 = pd.read_csv('SPRICE_Norwegian_Maritime_Data.csv')
dfc1 = df0.copy()
dfc1.head()

In [ ]:
# Create new df with variables we want to work with:
new_cols = ['TIMESTAMP', 'Air_temp_Act', 'Rel_Humidity_act', 'Rel_Air_Pressure', 'Wind_Speed_avg', 'Precipitation_Type', 'Precipitation_Intensity']

df = df0[new_cols]

df[df.isnull().any(axis=1)] # any missing data in columns
df.isnull().any()
df.head()

In [ ]:
df.info()

In [ ]:
# Checking the unique values in the 'weather' column
unique_fields = df['Precipitation_Type'].unique()
print(unique_fields)

In [ ]:
num_stdv = 1

# Define the labels dictionary
labels = {
    'Air_temp_Act': ['low', 'moderate', 'high'],
    'Rel_Humidity_act': ['low', 'moderate', 'high'],
    'Rel_Air_Pressure': ['low', 'moderate', 'high'],
    'Wind_Speed_avg': ['low', 'moderate', 'high'],
    # 'Precipitation_Type': ['0', '60', '70'],
    'Precipitation_Intensity': ['low', 'moderate', 'high']
}

# Create bounds for continuous labels
for col in df.columns:
    if col in labels:
        # print(col)
        col_mean = df[col].mean()
        col_stvd = df[col].std()
        lower = col_mean - col_stvd * num_stdv
        upper = col_mean + col_stvd * num_stdv
        bins = [-float('inf'), lower, upper, float('inf')]
        df.loc[:, col] = pd.cut(df[col], bins=bins, labels=labels[col]) # Wrong?


df.head()

In [ ]:
# Define the hierarchy
sprice_model = BayesianNetwork([
       ('Precipitation_Type', 'Precipitation_Intensity'), ('Precipitation_Type', 'Wind_Speed_avg'),
       ('Precipitation_Intensity', 'Rel_Air_Pressure'), ('Precipitation_Intensity', 'Rel_Humidity_act'),
       ('Wind_Speed_avg', 'Rel_Air_Pressure'), ('Wind_Speed_avg', 'Rel_Humidity_act'),
       ('Rel_Air_Pressure', 'Air_temp_Act'),
       ('Rel_Humidity_act', 'Air_temp_Act')
])

# And, the states for each variables
Precipitation_Type_states = ['0', '60', '70']
Precipitation_Intensity_states = ['low', 'mid', 'high']
Rel_Air_Pressure_states = ['low', 'mid', 'high']
Wind_Speed_avg_states = ['low', 'mid', 'high']
Rel_Humidity_act_states = ['low', 'mid', 'high']
Air_temp_Act_states = ['low', 'mid', 'high']

In [ ]:
# Calculate Probabilities

# Weather does not have any parents so all we need are the marginal probabilities of observing each weather type
weather_marginal = (df['Precipitation_Type'].value_counts()/len(df['Precipitation_Type'])).round(3)
weather_marginal = np.array([[value] for value in weather_marginal])


# Joint Propabilities
# Create dict where key=parent, value=child
var_dict = {
           ('Precipitation_Type',): ['Precipitation_Intensity', 'Wind_Speed_avg'], # Precipitation_Type
           ('Precipitation_Intensity', 'Wind_Speed_avg'): ['Rel_Air_Pressure'], # Precipitation_Intensity / Wind_Speed_avg
           ('Wind_Speed_avg', 'Precipitation_Intensity'): ['Rel_Humidity_act'], # Wind_Speed_avg / Precipitation_Intensity
           ('Rel_Air_Pressure', 'Rel_Humidity_act'): ['Air_temp_Act'], # Rel_Air_Pressure / Rel_Humidity_act
           }

# Create conditional distributions and store results in a list
cpd_lst = []
for key, value in var_dict.items():
   length = len(value)
   for i in range(length):
       value_given_key = df.groupby(list(key))[value[i]].value_counts(normalize=True).sort_index()
       cpd = value_given_key.unstack(fill_value=0).to_numpy().T
       cpd_lst.append(cpd)


# Note that we get 3 Nan values in the above conditional distributions. This is because one of the type of precipitation (low) did not contain any relation with temp_max.
# Therefore, normalization, does not produce the intended result.
# To mitigate this, we replace Nan with the equal probability within the three values, i.e., 0.33
for cpd in cpd_lst:
    for i in range(len(cpd[0])):
        col = cpd[:,i]
        if np.array_equal(col, np.array([0., 0., 0.])):
            cpd[:,i] = .33

# print(cpd_lst)

In [ ]:
# Creating tabular conditional probability distribution
Precipitation_Type_cpd = TabularCPD(variable='Precipitation_Type', 
                                    variable_card=3, 
                                    values=weather_marginal, 
                                    state_names={'Precipitation_Type': Precipitation_Type_states})

Precipitation_Intensity_cpd = TabularCPD(variable='Precipitation_Intensity', 
                                         variable_card=3, 
                                         evidence=['Precipitation_Type'], 
                                         evidence_card=[3],
                                         values=cpd_lst[0], 
                                         state_names={'Precipitation_Intensity': Precipitation_Intensity_states, 'Precipitation_Type': Precipitation_Type_states})

Wind_Speed_avg_cpd = TabularCPD(variable='Wind_Speed_avg', 
                                variable_card=3, 
                                evidence=['Precipitation_Type'], 
                                evidence_card=[3],
                                values=cpd_lst[1], 
                                state_names={'Wind_Speed_avg': Wind_Speed_avg_states, 'Precipitation_Type': Precipitation_Type_states})

Rel_Air_Pressure_cpd = TabularCPD(variable='Rel_Air_Pressure', 
                                  variable_card=3, 
                                  evidence=['Precipitation_Intensity', 'Wind_Speed_avg'], 
                                  evidence_card=[3, 3],
                                  values=cpd_lst[2], 
                                  state_names={'Rel_Air_Pressure': Rel_Air_Pressure_states, 'Precipitation_Intensity': Precipitation_Intensity_states, 'Wind_Speed_avg': Wind_Speed_avg_states})

Rel_Humidity_act_cpd = TabularCPD(variable='Rel_Humidity_act', 
                                  variable_card=3, 
                                  evidence=['Wind_Speed_avg', 'Precipitation_Intensity'], 
                                  evidence_card=[3, 3],
                                  values=cpd_lst[3], 
                                  state_names={'Rel_Humidity_act': Rel_Humidity_act_states, 'Wind_Speed_avg': Wind_Speed_avg_states, 'Precipitation_Intensity': Precipitation_Intensity_states})

Air_temp_Act_cpd = TabularCPD(variable='Air_temp_Act', 
                              variable_card=3, 
                              evidence=['Rel_Air_Pressure', 'Rel_Humidity_act'], 
                              evidence_card=[3, 3],
                              values=cpd_lst[4], 
                              state_names={'Air_temp_Act': Air_temp_Act_states, 'Rel_Air_Pressure': Rel_Air_Pressure_states, 'Rel_Humidity_act': Rel_Humidity_act_states})

In [ ]:
# Add CPDs and factors to the model
sprice_model.add_cpds(Precipitation_Type_cpd, Precipitation_Intensity_cpd, Wind_Speed_avg_cpd, Air_temp_Act_cpd, Rel_Air_Pressure_cpd, Rel_Humidity_act_cpd)
# Check if model is consistent
sprice_model.check_model()

In [ ]:
# Viewing nodes of the model
sprice_model.nodes()

In [ ]:
# Viewing edges of the model
sprice_model.edges()

In [ ]:
# Print the probability table of the weather node
print(Precipitation_Type_cpd)

# Print the probability table of the wind node
print(Precipitation_Intensity_cpd)

## Task 1.2

In [ ]:
from pgmpy.inference import VariableElimination

inference = VariableElimination(sprice_model)

In [ ]:
# Question 1:

# (a) What is the probability of the Actual air temperature when the Type of precipitation is 70?
phi_query = inference.query(variables=['Air_temp_Act'], evidence={'Precipitation_Type' : '70'})
print("Probability of the Actual air temperature when the Type of precipitation is 70:")
print(phi_query)

print()

# (b) What is the probability of the Type of precipitation when the Actual air temperature is high?
phi_query = inference.query(variables=['Precipitation_Type'], evidence={'Air_temp_Act' : 'high'})
print("Probability of the Type of precipitation when the Actual air temperature is high:")
print(phi_query)

In [ ]:
# Question 2:
# (a) Calculate all the possible joint probability and determine the best probable condition. Explain your results?

# Returns the highest probable state in the joint distribution of variables.
phi_query = inference.map_query(variables=['Air_temp_Act', 'Rel_Humidity_act', 'Rel_Air_Pressure', 'Wind_Speed_avg', 'Precipitation_Type', 'Precipitation_Intensity'], show_progress=False)
print("The most probable condition:")
print(phi_query)

print()
    
# (b) What is the most probable condition for Precipitation_Type, Precipitation_Intensity, and Wind_Speed_avg, combined?

# Returns the highest probable state in the joint distribution of variables.
phi_query = inference.map_query(variables=['Precipitation_Type', 'Precipitation_Intensity', 'Wind_Speed_avg'], show_progress=False)
print("The most probable condition for Precipitation_Type, Precipitation_Intensity, and Wind_Speed_avg:")
print(phi_query)



In [ ]:
# Question 3. Find the probability associated with each Type of precipitation, given that the precipitation intensity is medium? Explain your result.
phi_query = inference.query(variables=['Precipitation_Type'], evidence={'Precipitation_Intensity' : 'mid'})

print("The probability associated with each Type of precipitation, given that the precipitation intensity is medium:")
print(phi_query)


In [ ]:
# Question 4. What is the probability of each Type of precipitation given that precipitation intensity is medium and Average wind speed is low or medium? 
# Explain your method and results. How does the result change with the addition of wind factor compared to question 3 of Task 1.2?
phi_query = inference.query(variables=['Precipitation_Type'], evidence={'Precipitation_Intensity' : 'mid', 'Wind_Speed_avg' : 'low', 'Wind_Speed_avg' : 'mid'})

print("The probability of each Type of precipitation given that precipitation intensity is medium and Average wind speed is low or medium:")
print(phi_query)


# Task 1.3 - Approximate Inference

In [ ]:
from pgmpy.factors.discrete import State
from pgmpy.sampling import BayesianModelSampling

inference = BayesianModelSampling(sprice_model)

In [ ]:
# Repeat Q.1. (a) of Task 1.2 - What is the probability of high Average wind speed when the type of precipitation is 60?
evidence = [State('Precipitation_Type', '60')]

# Generates weighted sample(s) from joint distribution of the Bayesian Network, that comply with the given evidence
weighted_sample = inference.likelihood_weighted_sample(evidence=evidence, size=100000, show_progress=False)

weighted_probability = weighted_sample['Wind_Speed_avg'].value_counts(normalize=True)['high']
print("The probability of high Average wind speed when the type of precipitation is 60: ", weighted_probability)

In [ ]:
# Repeat Q.1. (b) of Task 1.2 - What is the probability of the type of precipitation being 60 when the Average wind speed is high?
evidence = [State('Wind_Speed_avg', 'mid')]

# Generates weighted sample(s) from joint distribution of the Bayesian Network, that comply with the given evidence
weighted_sample = inference.likelihood_weighted_sample(evidence=evidence, size=100000, show_progress=False)

weighted_probability = weighted_sample['Precipitation_Type'].value_counts(normalize=True)['60']
print("The probability of the type of precipitation being 60 when the Average wind speed is high: ", weighted_probability)

## Rejection Sampling

In [ ]:
# Repeat Q.2 . (a) of Task 1.2 - Calculate all the possible joint probability and determine the best probable condition. Explain your results?
rejection_sample = inference.rejection_sample(size=10000, show_progress=False)

rejection_counts = rejection_sample.value_counts(normalize=True)

conditions = rejection_counts.idxmax()

condition = {}
for index, node in enumerate(sprice_model.nodes()):
    condition[node] = conditions[index]

print("The most probable condition", condition)

In [ ]:
# Repeat Q.2 . (b) of Task 1.2 - What is the most probable condition for Precipitation_Type, Precipitation_Intensity, and Wind_Speed_avg, combined?
combination = ['Precipitation_Type', 'Precipitation_Intensity', 'Wind_Speed_avg']
rejection_sample = inference.rejection_sample(size=10000, show_progress=False)[combination]

rejection_counts = rejection_sample.value_counts(normalize=True)

conditions = rejection_counts.idxmax()

condition = {}
for index, node in enumerate(combination):
    condition[node] = conditions[index]

print("The most probable condition for Precipitation_Type, Precipitation_Intensity, and Wind_Speed_avg", condition)

## Approx Inference

In [ ]:
from pgmpy.inference import ApproxInference

inference = ApproxInference(sprice_model)

In [ ]:
# Repeat Q.3 of Task 1.2 - Find the probability associated with each type of precipitation, given that the precipitation intensity is medium? 
# Explain your result.
phi_query = inference.query(variables=['Precipitation_Type'], evidence={'Precipitation_Intensity' : 'mid'}, show_progress=False)

print("he probability associated with each type of precipitation, given that the precipitation intensity is medium")
print(phi_query)


# Normal Sampling

In [ ]:
# Repeat Q.4 of Task 1.2 - 
# What is the probability of each type of precipitation given that precipitation intensity is medium and average wind speed is low or medium? 
# Explain your method and results. How does the result change with the addition of wind factor compared to question 3 of Task 1.2?
inference = BayesianModelSampling(sprice_model)

forward_sample = inference.forward_sample(size=10000, show_progress=False)

conditioned_samples = forward_sample[(forward_sample['Precipitation_Intensity'] == 'mid') & (forward_sample['Wind_Speed_avg'].isin(['low', 'mid']))]

weather_probabilities = conditioned_samples['Precipitation_Type'].value_counts(normalize=True)

print("The probability of each type of precipitation given that precipitation intensity is medium and average wind speed is low or medium")
print(weather_probabilities)